# Day-ahead Energy Production Data Visualization App

This Dash application allows users to visualize energy production and consumption data by fetching it from the ENTSO-E API. The application provides input fields for users to enter their API key, country code, and select a date for which they want to retrieve data.

In [5]:
import pandas as pd
import pytz  # Importing pytz for timezone management
from dash import Dash, dcc, html  # Importing necessary components from Dash
from dash.dependencies import Input, Output, State  # Importing dependencies for callbacks
import plotly.graph_objects as go  # Importing Plotly for graphing
import plotly.express as px  # Importing Plotly Express for easier visualization
from entsoe import EntsoePandasClient  # Importing the ENTSO-E API client

# Initialize the Dash app
app = Dash(__name__)

# Define a color palette for different energy production and consumption types
color_palette = {
    'Biomass Generation': 'green',
    'Biomass Consumption': 'lightgreen',
    'Fossil Brown coal/Lignite Generation': 'saddlebrown',
    'Fossil Brown coal/Lignite Consumption': 'tan',
    'Fossil Gas Generation': 'orange',
    'Fossil Gas Consumption': 'peachpuff',
    'Fossil Hard coal Generation': 'darkred',
    'Fossil Hard coal Consumption': 'lightcoral',
    'Fossil Oil Generation': 'darkblue',
    'Fossil Oil Consumption': 'lightblue',
    'Geothermal Generation': 'lightgreen',
    'Geothermal Consumption': 'palegreen',
    'Hydro Pumped Storage Generation': 'royalblue',
    'Hydro Pumped Storage Consumption': 'darkgrey',
    'Hydro Run-of-river and poundage Generation': 'deepskyblue',
    'Hydro Run-of-river and poundage Consumption': 'skyblue',
    'Hydro Water Reservoir Generation': 'aqua',
    'Hydro Water Reservoir Consumption': 'aliceblue',
    'Nuclear Generation': 'yellow',
    'Nuclear Consumption': 'lightyellow',
    'Other Generation': 'gray',
    'Other Consumption': 'lightgray',
    'Other renewable Generation': 'lime',
    'Other renewable Consumption': 'limegreen',
    'Solar Generation': 'gold',
    'Solar Consumption': 'lightgoldenrodyellow',
    'Waste Generation': 'darkkhaki',
    'Waste Consumption': 'khaki',
    'Wind Offshore Generation': 'purple',
    'Wind Offshore Consumption': 'thistle',
    'Wind Onshore Generation': 'wheat',
    'Wind Onshore Consumption': 'lavender'
}

# Define CSS styling for the input fields and submit button
INPUT_STYLE = {
    'padding': '10px',
    'margin': '5px',
    'width': '100%',
    'border-radius': '5px',
    'border': '1px solid #ccc',
    'font-size': '16px'
}

BUTTON_STYLE = {
    'padding': '10px 20px',
    'margin': '10px',
    'background-color': '#4CAF50',
    'color': 'white',
    'border': 'none',
    'border-radius': '5px',
    'cursor': 'pointer',
    'font-size': '16px',
    'width': '100%',
    'text-align': 'center',
    'transition': 'background-color 0.3s'
}

DATE_PICKER_STYLE = {
    'padding': '10px',
    'margin': '5px',
    'font-size': '16px',
    'border-radius': '5px',
    'width': '100%',
    'background-color': 'white',
    'border': '1px solid #ccc'
}

# Define the layout of the app
app.layout = html.Div([
    html.H1("Energy Production Data", style={'text-align': 'center'}),  # Title of the app
    
    # Input fields for user data
    html.Div([
        html.Label('Enter your API Key:', style={'font-weight': 'bold'}),
        dcc.Input(id='api-key', type='text', placeholder='Enter API Key', style=INPUT_STYLE),  # API Key input
    ], style={'width': '30%', 'margin': '0 auto'}),  # Center the input field

    html.Div([
        html.Label('Select Country Code:', style={'font-weight': 'bold'}),
        dcc.Input(id='country-code', type='text', placeholder='e.g., DE_LU', style=INPUT_STYLE),  # Country code input
    ], style={'width': '30%', 'margin': '0 auto'}),  # Center the input field
     
    # DatePicker for selecting a date
    html.Div([
        html.Label('Select a Date:', style={'font-weight': 'bold'}),
        dcc.DatePickerSingle(
            id='date-picker',
            min_date_allowed=pd.Timestamp(2015, 1, 1),  # Minimum date allowed
            max_date_allowed=pd.Timestamp.now(),  # Maximum date allowed
            initial_visible_month=pd.Timestamp.now(),  # Initial month displayed
            date=pd.Timestamp.now().date(),  # Default date selected
            display_format='DD.MM.YYYY',  # Date format displayed
            style=DATE_PICKER_STYLE  # Apply styles to DatePicker
        ),
    ], style={'width': '30%', 'margin': '0 auto'}),  # Center the DatePicker

    # Submit button with consistent style
    html.Div([
        html.Button('Submit', id='submit-button', style=BUTTON_STYLE),  # Submit button
    ], style={'width': '30%', 'margin': '0 auto'}),  # Center the button

    # Output Graph
    dcc.Graph(id='generation-graph'),  # Graph component for displaying the data
])

# Callback for fetching data and updating the graph
@app.callback(
    Output('generation-graph', 'figure'),  # Output to the generation graph
    [Input('submit-button', 'n_clicks')],  # Input triggered by the submit button
    [State('api-key', 'value'),  # State to hold the API key
     State('country-code', 'value'),  # State to hold the country code
     State('date-picker', 'date')]  # State to hold the selected date
)
def update_graph(n_clicks, api_key, country_code, selected_date):
    """Fetch data from ENTSO-E API and update the graph based on user input."""
    
    # Return an empty figure if the API key or country code is missing
    if not api_key or not country_code:
        return go.Figure()

    try:
        # Initialize the EntsoePandasClient with the provided API key
        client = EntsoePandasClient(api_key=api_key)

        # Convert selected_date to a Pandas timestamp
        selected_date = pd.to_datetime(selected_date)

        # Define the timezone as Europe/Brussels (CET)
        cet = pytz.timezone('Europe/Brussels')

        # Get the start and end time for the selected date (24-hour window)
        start = cet.localize(pd.Timestamp(selected_date).replace(hour=0, minute=0, second=0, microsecond=0))
        end = start + pd.Timedelta(days=1)

        # Fetch generation data for the selected country and date from the ENTSO-E API
        generation_data = client.query_generation(country_code, start=start, end=end)

        # Format column names (assuming multi-level column names)
        generation_data.columns = [
            ' '.join(col).strip().replace('Actual Aggregated', 'Generation').replace('Actual Consumption', 'Consumption')
            for col in generation_data.columns.values
        ]

        # Resample the data to 15-minute intervals
        generation_data_resampled = generation_data.resample('15min').sum()

        # Convert DataFrame to long format for easier plotting
        df_melted = generation_data_resampled.reset_index().melt(id_vars='index', var_name='Production Type', value_name='Generation (MW)')

        # Convert consumption values to negative for visualization
        df_melted['Generation (MW)'] = df_melted.apply(
            lambda row: -row['Generation (MW)'] if 'Consumption' in row['Production Type'] else row['Generation (MW)'], axis=1)

        # Create a stacked bar plot using Plotly
        fig = go.Figure()

        # Define a list of colors from the Viridis palette
        #color_scale = px.colors.sequential.Viridis
        
        # Loop through each production type and add it to the figure
        for i, production_type in enumerate(df_melted['Production Type'].unique()):
            df_filtered = df_melted[df_melted['Production Type'] == production_type]
            fig.add_trace(
                go.Bar(
                    x=df_filtered['index'],
                    y=df_filtered['Generation (MW)'],
                    name=production_type,
                    hovertemplate=('%{fullData.name}: %{y:.0f}<extra></extra>'),
                    #width=900_000,  # Bar width to spread out bars (milliseconds)
                    marker_color=color_palette.get(production_type, 'gray'), #color_scale[i % len(color_scale)],
                    hoverinfo='text'
                )
            )

        # Customize the layout of the plot
        fig.update_layout(
            showlegend=False,
            barmode="relative",
            title=f"<b>Actual Generation per Production Type - {country_code} on {selected_date.strftime('%Y-%m-%d')}</b>",
            title_x=0.5,
            hovermode="x",
            plot_bgcolor="white",
            xaxis=dict(
                showspikes=True, 
                spikecolor='black', 
                spikemode='across',
                spikesnap='cursor', 
                spikethickness=2,
                tickformat='%H:%M',
                #tickvals=pd.date_range(start=start, end=end, freq='2h'),
                #ticktext=pd.date_range(start=start, end=end, freq='2h').hour,
                range=[start, end],
                tickangle=-45,
                linecolor='black',
                ticks='outside',
                title="Time [Hours]"
            ),
            yaxis=dict(
                showline=True,
                linewidth=2,
                linecolor='black',
                title='Net Generation [MW]',
                title_font=dict(color='black'),
                tickformat='{%000}',
                #tickvals=list(range(-10000, 100000, 5000)),
                #ticktext=[f"{i}" for i in range(-10000, 100000, 5000)],
                showgrid=True,  
                gridcolor='lightgray',  
                gridwidth=1
            ),
            height=800
        )

        return fig

    except Exception as e:
        print(f"Error fetching data: {e}")
        return go.Figure()  # Return empty figure in case of error

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True) # in case of error, add port=8051

![Actual Geberation per Production Type - DE_LU on 2023-07-15](images/generation_graph.png)